In [55]:
import geopandas as gpd
import pandas as pd

In [56]:
df_population = pd.read_csv(
    'Data/Population/census_proportionate_v5.csv', encoding='ISO-8859-1')

In [57]:
df_population.columns

Index(['CODEMAMROT', 'NOM', 'YEAR', 'POPULATION'], dtype='object')

In [58]:
gdf_boroughs = gpd.read_file('Data/Boroughs/LIMADMIN.shp')


In [59]:
gdf_boroughs.columns

Index(['MUNID', 'CODEID', 'CODEMAMROT', 'NOM', 'TYPE', 'ABREV', 'NUM', 'AIRE',
       'PERIM', 'geometry'],
      dtype='object')

### Population total per borough or affiliated city and year

In [60]:
df_population.head()

,CODEMAMROT,NOM,YEAR,POPULATION
0,REM23,Ahuntsic-Cartierville,2011,126891.0
1,REM09,Anjou,2011,41928.0
2,REM34,CÃ´te-des-Neiges-Notre-Dame-de-GrÃ¢ce,2011,165031.0
3,REM27,Lachine,2011,41616.0
4,REM17,LaSalle,2011,74276.0


In [61]:
df_population = df_population.rename(
    columns={'CODEMARMOT': 'CODEMAMROT'})

In [62]:
df_grouped_population = df_population.groupby(['NOM', 'CODEMAMROT', 'YEAR'])[
    'POPULATION'].sum().rename('POPULATION_SUM').reset_index()
print(df_grouped_population)

                       NOM CODEMAMROT  YEAR  POPULATION_SUM
0    Ahuntsic-Cartierville      REM23  2011        126891.0
1    Ahuntsic-Cartierville      REM23  2012        128361.8
2    Ahuntsic-Cartierville      REM23  2013        129832.6
3    Ahuntsic-Cartierville      REM23  2014        131303.4
4    Ahuntsic-Cartierville      REM23  2015        132774.2
..                     ...        ...   ...             ...
437              Westmount      66032  2019         19919.6
438              Westmount      66032  2020         19788.8
439              Westmount      66032  2021         19658.0
440              Westmount      66032  2022         19527.2
441              Westmount      66032  2023         19396.4

[442 rows x 4 columns]


In [63]:
df_grouped_population = df_grouped_population.rename(
    columns={'borough': 'NOM', 'year': 'YEAR'})
df_grouped_population.columns

Index(['NOM', 'CODEMAMROT', 'YEAR', 'POPULATION_SUM'], dtype='object')

### Population density per borough or affiliated city and year

In [64]:
df_grouped_population = df_grouped_population.merge(gdf_boroughs,
                                                    on='CODEMAMROT',
                                                    how='left')

In [65]:
df_grouped_population.head()

,NOM_x,CODEMAMROT,YEAR,POPULATION_SUM,MUNID,CODEID,NOM_y,TYPE,ABREV,NUM,AIRE,PERIM,geometry
0,Ahuntsic-Cartierville,REM23,2011,126891.0,66023,4,Ahuntsic-Cartierville,Arrondissement,AC,24,2.557119e+07,37790.097,"POLYGON ((-73.76363 45.51193, -73.76354 45.511..."
1,Ahuntsic-Cartierville,REM23,2012,128361.8,66023,4,Ahuntsic-Cartierville,Arrondissement,AC,24,2.557119e+07,37790.097,"POLYGON ((-73.76363 45.51193, -73.76354 45.511..."
2,Ahuntsic-Cartierville,REM23,2013,129832.6,66023,4,Ahuntsic-Cartierville,Arrondissement,AC,24,2.557119e+07,37790.097,"POLYGON ((-73.76363 45.51193, -73.76354 45.511..."
3,Ahuntsic-Cartierville,REM23,2014,131303.4,66023,4,Ahuntsic-Cartierville,Arrondissement,AC,24,2.557119e+07,37790.097,"POLYGON ((-73.76363 45.51193, -73.76354 45.511..."
4,Ahuntsic-Cartierville,REM23,2015,132774.2,66023,4,Ahuntsic-Cartierville,Arrondissement,AC,24,2.557119e+07,37790.097,"POLYGON ((-73.76363 45.51193, -73.76354 45.511..."


In [66]:
df_grouped_population.shape

(442, 13)

In [67]:
df_grouped_population['POPULATION_DENSITY'] = df_grouped_population['POPULATION_SUM'] / \
    df_grouped_population['AIRE']

In [68]:
df_grouped_population.columns

Index(['NOM_x', 'CODEMAMROT', 'YEAR', 'POPULATION_SUM', 'MUNID', 'CODEID',
       'NOM_y', 'TYPE', 'ABREV', 'NUM', 'AIRE', 'PERIM', 'geometry',
       'POPULATION_DENSITY'],
      dtype='object')

In [69]:
df_grouped_population = df_grouped_population[['CODEMAMROT',
                                               'YEAR',
                                               'POPULATION_SUM',
                                               'AIRE',
                                               'POPULATION_DENSITY']]

In [70]:
df_grouped_population.describe(include='all')

,CODEMAMROT,YEAR,POPULATION_SUM,AIRE,POPULATION_DENSITY
count,442,442.000000,442.000000,4.420000e+02,442.000000
unique,34,NaN,NaN,NaN,NaN
top,REM23,NaN,NaN,NaN,NaN
freq,13,NaN,NaN,NaN,NaN
mean,NaN,2017.000000,57530.137104,1.817797e+07,0.003633
std,NaN,3.745897,48707.383334,1.194109e+07,0.002921
min,NaN,2011.000000,5.000000,1.805089e+05,0.000028
25%,NaN,2014.000000,19082.550000,8.151665e+06,0.000926
50%,NaN,2017.000000,43198.300000,1.618194e+07,0.003122
75%,NaN,2020.000000,91785.050000,2.519727e+07,0.005062


In [71]:
df_grouped_population['YEAR'] = df_grouped_population['YEAR'].astype('object')

In [72]:
df_grouped_population.dtypes

CODEMAMROT             object
YEAR                   object
POPULATION_SUM        float64
AIRE                  float64
POPULATION_DENSITY    float64
dtype: object

# Output file

In [74]:
df_grouped_population.to_csv(
    'Data/Processed_Datasets/Combined_tables/df_fe_population.csv')